In [1]:
import torch
from utils.fda import FDA_source_to_target
from utils.aug import ExtCompose, ExtTransforms, ExtRandomHorizontalFlip , ExtScale , ExtRandomCrop, ExtGaussianBlur, ExtColorJitter, ExtResize, ExtToV2Tensor
from datasets.cityscapes import CityScapes
from datasets.GTA5 import GTA5
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import random

## FDA Style Transfer Test

In [2]:
beta = 0.05
h = 512
w = 1024
b = (  np.floor(np.amin((h,w))*beta)  ).astype(int)
b

25

In [3]:
standard_transformations = ExtCompose([ExtScale(0.5,interpolation=Image.Resampling.BILINEAR), ExtToV2Tensor()])
transformations = ExtCompose([ExtResize((512,1024)), ExtToV2Tensor()])
target_transformations = standard_transformations

train_dataset = GTA5(root='dataset',transforms=transformations)
target_dataset_train = CityScapes(split='train',transforms=target_transformations)

source_dataloader_train = DataLoader(train_dataset,
                batch_size=1,
                shuffle=False,
                num_workers=1,
                pin_memory=False,
                drop_last=True)
target_dataloader_train = DataLoader(target_dataset_train,
                batch_size=1,
                shuffle=False,
                num_workers=1,
                pin_memory=False,
                drop_last=True)

for i, (source_data, target_data) in enumerate(zip(source_dataloader_train,target_dataloader_train)):
  
    if i > 5:
        break

    source_data,source_label = source_data
    target_data,_ = target_data
    source_data = source_data.cuda()
    source_label = source_label.long().cuda()
    target_data = target_data.cuda()

    t_source_data = FDA_source_to_target(source_data, target_data, beta=beta)

    # Visualize the image (.png) and the transformed image (.jpg)
    Image.fromarray(target_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()
    Image.fromarray(source_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()
    Image.fromarray(t_source_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()


root:  dataset


## Strong Data Augmentation Test

In [9]:
size = (512,1024)
transformations = ExtCompose([
                ExtScale(random.choice([0.75,1,1.25,1.5,1.75,2]),interpolation=Image.Resampling.BILINEAR),
                ExtRandomCrop(size),
                ExtRandomHorizontalFlip(),
                ExtGaussianBlur(p=0.5, radius=1),
                ExtColorJitter(p=0.5, brightness=0.2, contrast=0.1, saturation=0.1, hue=0.2),
                ExtToV2Tensor()])
train_dataset = GTA5(root='dataset',transforms=transformations)
source_dataloader_train = DataLoader(train_dataset,
                batch_size=1,
                shuffle=False,
                num_workers=1,
                pin_memory=False,
                drop_last=True)

for i, (source_data, source_label) in enumerate(source_dataloader_train):
  
    if i > 20:
        break

    print(f'Augmented image {i+1}')
    # Visualize the image (.png) and the transformed image (.jpg)
    Image.fromarray(source_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()


Augmented image 1
Augmented image 2
Augmented image 3
Augmented image 4
Augmented image 5
Augmented image 6
Augmented image 7
Augmented image 8
Augmented image 9
Augmented image 10
Augmented image 11
Augmented image 12
Augmented image 13
Augmented image 14
Augmented image 15
Augmented image 16
Augmented image 17
Augmented image 18
Augmented image 19
Augmented image 20
Augmented image 21
